# Project Name: Gender Wage Gap in Developing Countries


## This is a jupyter notebook to create wage/ income ratios similar to Table2


### Import packages pandas, numpy and read data (.csv file) as df (Data Frame) 

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('/Users/jugalmarfatia/Documents/summer2018/RA Work/ipumsi_00002.csv')

## First 5 rows to check dataset

In [2]:
df.head()

,COUNTRY,YEAR,SAMPLE,SERIAL,HHWT,PERNUM,PERWT,AGE,SEX,MARST,...,LIT,EDATTAIN,EDATTAIND,YRSCHOOL,EMPSTAT,EMPSTATD,LABFORCE,INCTOT,INCEARN,INCWAGE
0,76,2010,76201001,1000,18.54,1,18.54,22,1,2,...,2.0,2,212,NaN,1,110,2,1000.0,1000.0,NaN
1,76,2010,76201001,1000,18.54,2,18.54,17,2,2,...,2.0,2,212,NaN,3,300,1,0.0,99999999.0,NaN
2,76,2010,76201001,2000,20.23,1,20.23,54,1,2,...,1.0,1,110,NaN,1,110,2,700.0,700.0,NaN
3,76,2010,76201001,2000,20.23,2,20.23,44,2,2,...,2.0,1,120,NaN,1,110,2,419.0,250.0,NaN
4,76,2010,76201001,2000,20.23,3,20.23,11,1,1,...,2.0,1,130,NaN,3,300,9,0.0,99999999.0,NaN


### Print the list of countries codes (Brazil, India, Mexico) .

In [3]:
df['COUNTRY'].unique()


array([ 76, 214, 356, 388, 484, 591, 630, 710, 780, 858, 862])

### Convert '99999' to null.
- 999999 indicates missing or unknown thus setting them to null in order to not include them in calculations

In [4]:
df = df.replace(99999999.0,np.NaN)

# Function to calculate wage ratios
Below I am defining a function (method) which caculates men:women income ratios and outputs the below variables in a table. 

## Variable Name: Description 
- ***country_name***: Name of country 
- ***income_type***: Name of income varibale in Ipums
- ***lf_part_rate***: Overall labor force Participation rate
- ***men_lf_part_rate***: Men labor force Participation rate
- ***women_lf_part_rate***: Women labor force Participation rate
- ***year***: Year of dataset
- ***men_sample_size***: Number men in the data with income > 0 
- ***women_sample_size***: Number women in the data with income > 0
- ***men income mean (std)***: Average (std) income for men
- ***women income mean (std)***: Average (std) income for women
- ***mean_ratio***:	Ratio of women:men income at mean
- ***10_percentile***: Ratio of women:men income at 10th percentile 
- ***25_percentile***: Ratio of women:men income at 25th percentile 
- ***90_percentile***: Ratio of women:men income at 90th percentile 

## Procedure:
- Include only employed persons.
- Exclude persons with income = 0.
- Calculate based on person weight. 

In [8]:
import weightedcalcs as wc

def wage_ratio(df, country, country_name, var_name):
    # Keep only data for the country specified
    df = df.loc[df['COUNTRY'] == country].reset_index()
    
    # Labor force particiption rate of income > 0
    lf_part_rate = df.loc[df[var_name] > 0]["PERWT"].sum()/df["PERWT"].sum()    
    
    # Seperate men and women dataframe
    men = df.loc[df['SEX'] == 1]
    women = df.loc[df['SEX'] == 2]
    
    # Create output/ return table 
    d = {'income_type': [var_name], 'year': [round(df['YEAR'].mean(), 0)], 'country_name':
         [country_name], 'lf_part_rate': [lf_part_rate],}
    
    # Define weight variable
    calc = wc.Calculator("PERWT")
    
    # Make output table as pandas object
    final_df = pd.DataFrame(data=d)
    
    # Calculate men and women labor force participation rate
    final_df['men_lf_part_rate'] = men.loc[men[var_name] > 0]["PERWT"].sum()/men["PERWT"].sum()
    final_df['women_lf_part_rate'] = women.loc[women[var_name] > 0]["PERWT"].sum()/women["PERWT"].sum()
    
    # Keep only person with income > 0 for ratio calculation
    men = men.loc[df[var_name] > 0]
    women = women.loc[df[var_name] > 0]
    
    # Calculate sample size for men and women
    final_df['men_sample_size'] = men[var_name].count()
    final_df['women_sample_size'] = women[var_name].count()
    
    # Calculate mean and std of income
    final_df['men income: mean (std)'] = str(round(calc.mean(men, var_name))) + ' (' + str(round(calc.std(men, var_name))) + ')'
    final_df['women income: mean (std)'] = str(round(calc.mean(women, var_name))) + ' (' + str(round(calc.std(women, var_name))) + ')' 
    
    # Calculate ratios at different percentiles
    final_df['mean_ratio'] = round(calc.mean(women, var_name)/ calc.mean(men, var_name), 3)
    final_df['10_percentile'] = round(calc.quantile(women, var_name, 0.1) 
                                      / calc.quantile(men, var_name, 0.1) ,3)
    
    final_df['25_percentile'] = round(calc.quantile(women, var_name, 0.25) 
                                      / calc.quantile(men, var_name, 0.25) ,3)
    
    final_df['90_percentile'] = round(calc.quantile(women, var_name, 0.90) 
                                      / calc.quantile(men, var_name, 0.90) ,3)
    
    # return output as pandas table 
    return final_df



# Calculate ratios for each country by calling the above function and display results table. 


In [9]:
brazil_df = wage_ratio(df, 76, 'Brazil' ,'INCTOT')
india_df = wage_ratio(df, 356, 'India' ,'INCWAGE')
mexico_df = wage_ratio(df, 484, 'Mexico' ,'INCTOT')
dr_df = wage_ratio(df, 214, 'Dominican Republic' ,'INCTOT')
jamaica_df = wage_ratio(df, 388, 'Jamaica' ,'INCWAGE')
panama_df = wage_ratio(df, 591, 'Panama' ,'INCTOT')
puerto_rico_df = wage_ratio(df, 630, 'Puerto Rico' ,'INCTOT')
sa_df = wage_ratio(df, 710, 'South Africa' ,'INCTOT')
tnt_df = wage_ratio(df, 780, 'Trinadad and Tabago' ,'INCTOT')
uruguay_df = wage_ratio(df, 858, 'Uruguay' ,'INCWAGE')
venezuela_df = wage_ratio(df, 862, 'venezuela' ,'INCTOT')


combined= pd.concat([brazil_df, india_df, mexico_df, dr_df, jamaica_df, panama_df
                     , puerto_rico_df, tnt_df, uruguay_df, venezuela_df, sa_df], ignore_index=True)
combined

,income_type,year,country_name,lf_part_rate,men_lf_part_rate,women_lf_part_rate,men_sample_size,women_sample_size,men income: mean (std),women income: mean (std),mean_ratio,10_percentile,25_percentile,90_percentile
0,INCTOT,2010.0,Brazil,0.721546,0.762668,0.682094,3627618,3346584,2056015.0 (4039851.0),2127567.0 (4091542.0),1.035,0.366,1.000,1.000
1,INCWAGE,2004.0,India,0.164715,0.233900,0.091966,65712,21276,682.0 (1085.0),350.0 (609.0),0.513,0.533,0.519,0.355
2,INCTOT,2000.0,Mexico,0.677122,0.796388,0.563845,3848028,2913775,3947325.0 (4885709.0),5003819.0 (4998748.0),1.268,0.439,0.875,1.000
3,INCTOT,2002.0,Dominican Republic,0.961784,0.967082,0.956529,413009,411823,6668193.0 (4711841.0),7874209.0 (4090263.0),1.181,1.667,2500.000,1.000
4,INCWAGE,2001.0,Jamaica,1.000000,1.000000,1.000000,99662,105517,7171301.0 (4418506.0),7973007.0 (3953872.0),1.112,1.000,24.038,1.000
5,INCTOT,2010.0,Panama,0.546752,0.630052,0.462717,107933,78574,6107.0 (22873.0),5452.0 (21791.0),0.893,0.800,0.667,0.759
6,INCTOT,2010.0,Puerto Rico,0.784060,0.815178,0.755377,14068,14647,2593754.0 (4365671.0),2424429.0 (4271950.0),0.935,0.725,0.733,1.000
7,INCTOT,2000.0,Trinadad and Tabago,0.755126,0.856449,0.654380,47753,36695,4590251.0 (4981801.0),5379356.0 (4984686.0),1.172,0.969,0.840,1.000
8,INCWAGE,2006.0,Uruguay,0.292749,0.338738,0.251785,41093,30436,7788.0 (8594.0),5695.0 (6003.0),0.731,0.545,0.686,0.773
9,INCTOT,2001.0,venezuela,0.624349,0.734329,0.516779,837478,602576,4706221.0 (4991213.0),6012203.0 (4896375.0),1.278,1.200,1.250,1.000
